# Suport Vector Machine

https://fr.wikipedia.org/wiki/Machine_%C3%A0_vecteurs_de_support

Exemple d'un problème de discrimination à deux classes, avec une séparatrice linéaire : la droite d'équation y = x. Le problème est linéairement séparable :

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Separatrice_lineaire.svg/525px-Separatrice_lineaire.svg.png)

Mais c'est rare en pratique, d'où l'astuce du noyau (en anglais kernel trick), qui consiste à reconsidérer le problème dans un espace de dimension supérieure. Par exemple :

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Separatrice_non_lineaire.svg/525px-Separatrice_non_lineaire.svg.png)

![](https://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d_hyperplane.png)

**Note** Code MLLib et pas la version plus moderne ML

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('SVM') \
    .getOrCreate()
sc = spark.sparkContext

In [4]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("/data/mllib/sample_svm_data.txt")
parsedData = data.map(parsePoint)


In [5]:
data.take(3)

['1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0',
 '0 2.857738033247042 0 0 2.619965104088255 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0',
 '0 2.857738033247042 0 2.061393766919624 0 0 2.004684436494304 0 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0']

In [7]:

# Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
error_nb = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count()
trainErr = error_nb / float(parsedData.count())
print("Training Error = " + str(trainErr))


Training Error = 0.38198757763975155


In [8]:
# Save and load model
# model.save(sc, "target/tmp/pythonSVMWithSGDModel")
# sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

In [9]:
sc.stop()